In [ ]:
# Read in files
# note that we only keep DISTINCT (i.e. unique) person IDs from here on out
for i in range(0,10):
    infile = (f'circuit_data/zips_circuit_criminal_201{i}.csv')
    spark = SparkSession \
        .builder \
        .appName("Racial Disparity") \
        .getOrCreate()
    circuit = spark.read.csv(infile, inferSchema = True, header = True)
    circuit.createOrReplaceTempView("data")
    sqlDF = spark.sql(
        
        # currently set to look at marijuana code sections only
        # but just take out code sections to see everything (or add more code sections)
        '''
        SELECT COUNT(DISTINCT(person_id)) as Count, Race, Sex, fips FROM data
                   WHERE (CodeSection LIKE '18.2-248%' 
                   OR CodeSection LIKE '18.2-250%'
                   OR CodeSection LIKE '18.2-255%')
                   GROUP BY Race, fips, Sex
                   ORDER BY Count DESC
                   '''
    )
    data = sqlDF.toPandas()
    data['Race and Sex'] = data['Race'] + ' - ' + data['Sex']
    data = data[['fips', 'Race and Sex', 'Count']]
    df_p = data.pivot_table(index=['fips'], columns=['Race and Sex'], values='Count', aggfunc=np.sum)
    data_1 = df_p.fillna(0)
    data_1.to_csv(f'circuit_data/201{i}_finished.csv')

# ================
# NOTE: Add/remove as desired to change the type of query
# Marijuana
# WHERE (CodeSection LIKE '18.2-248%' 
# OR CodeSection LIKE '18.2-250%'
# OR CodeSection LIKE '18.2-255%')

# Felonys
# AND (ChargeType == 'Felony')

In [ ]:
# Some code to rename 2019 to match 2010-2018 column labels (currently not using)
x = pd.read_csv('Data/2019_finished.csv')
new_cols = {'American Indian Or Alaskan Native - Female':         'American Indian - Female',
            'American Indian Or Alaskan Native - Male':           'American Indian - Male',
            'Asian Or Pacific Islander - Female':                 'Asian Or Pacific Islander - Female',
            'Asian Or Pacific Islander - Male':                   'Asian Or Pacific Islander - Male',
            'Black - Female':                                     'Black (Non-Hispanic) - Female',
            'Black - Male':                                       'Black (Non-Hispanic) - Male',
            'Hispanic - Female':                                  'Hispanic - Female',
            'Hispanic - Male':                                    'Hispanic - Male',
            'White - Female':                                     'White Caucasian (Non-Hispanic) - Female',
            'White - Male':                                       'White Caucasian (Non-Hispanic) - Male',
            'Other (Includes Not Applicable, Unknown) - Female':  'Other (Includes Not Applicable, Unknown) - Female',
            'Other (Includes Not Applicable, Unknown) - Male':    'Other (Includes Not Applicable, Unknown) - Male',
           }
x.rename(columns=new_cols,
          inplace=True)
x.to_csv('Data/2019_finished.csv', index=None)

In [ ]:
for i in range(0,10):
    # Note: make sure to reset the year each time
    infile = 'Data/census_data_fips.csv'
    spark = SparkSession \
    .builder \
    .appName("Census Data") \
    .getOrCreate()
    circuit = spark.read.csv(infile, inferSchema=True, header = True)
    circuit.createOrReplaceTempView("census_data")
    sqlDF = spark.sql(f'SELECT * \
                   FROM census_data \
                   WHERE Year = 201{i}')
    census = sqlDF.toPandas()
    
    #--------------------#
    data = pd.read_csv(f'Data/201{i}_finished.csv')
    #--------------------#
    data['Total_Crimes'] = 0
    data['Total_Crimes'] = data.sum(axis=1) 
    data.head(2)
    #del census['Year']
    census.head(2)
    result = pd.merge(census,
                  data,
                  on='fips', 
                  how='left')
    result.head(2)
    data.columns
    result['Percent_Overall']          = (result['Total_Crimes']   / result['Total'])*100
    result['Percent_White_Male']       = (result['White Caucasian (Non-Hispanic) - Male']   / result['White_Male'])*100
    result['Percent_White_Female']     = (result['White Caucasian (Non-Hispanic) - Female'] / result['White_Female'])*100
    result['Percent_Black_Male']       = (result['Black (Non-Hispanic) - Male']             / result['Black_Male'])*100
    result['Percent_Black_Female']     = (result['Black (Non-Hispanic) - Female']           / result['Black_Female'])*100
    result['Percent_Nat_Amer_Male']    = (result['American Indian - Male']                  / result['Native_Amer_Male'])*100
    #result['Percent_Nat_Amer_Female']  = (result['American Indian - Female']                / result['Native_Amer_Female'])*100
    result['Percent_Hispanic_Male']    = (result['Hispanic - Male']                         / result['Hispanic_Male'])*100
    result['Percent_Hispanic_Female']  = (result['Hispanic - Female']                       / result['Hispanic_Female'])*100
    result['Percent_Asian_Pac_Male']   = (result['Asian Or Pacific Islander - Male']        / result['Asian_Male'])*100
    result['Percent_Asian_Pac_Female'] = (result['Asian Or Pacific Islander - Female']      / result['Asian_Female'])*100
    
    result['Percent_Overall']            = round(result['Percent_Overall'],3)
    result['Disparity_White_Male']       = round(result['Percent_White_Male'] - result['Percent_Overall'],3)
    result['Disparity_White_Female']     = round(result['Percent_White_Female'] - result['Percent_Overall'],3)
    result['Disparity_Black_Male']       = round(result['Percent_Black_Male'] - result['Percent_Overall'],3)
    result['Disparity_Black_Female']     = round(result['Percent_Black_Female'] - result['Percent_Overall'],3)
    result['Disparity_Nat_Amer_Male']    = round(result['Percent_Nat_Amer_Male'] - result['Percent_Overall'],3)
    #result['Disparity_Nat_Amer_Female']  = round(result['Percent_Nat_Amer_Female'] - result['Percent_Overall'],3)
    result['Disparity_Hispanic_Male']    = round(result['Percent_Hispanic_Male'] - result['Percent_Overall'],3)
    result['Disparity_Hispanic_Female']  = round(result['Percent_Hispanic_Female'] - result['Percent_Overall'],3)
    result['Disparity_Asian_Pac_Male']   = round(result['Percent_Asian_Pac_Male'] - result['Percent_Overall'],3)
    result['Disparity_Asian_Pac_Female'] = round(result['Percent_Asian_Pac_Female'] - result['Percent_Overall'],3)
    
    result = result[['fips', 
                     'CTYNAME',
                     'Percent_Overall',
                     'Disparity_White_Male',
                     'Disparity_White_Female',
                     'Disparity_Black_Male',
                     'Disparity_Black_Female', 
                     'Disparity_Nat_Amer_Male',
                     #'Disparity_Nat_Amer_Female', 
                     'Disparity_Hispanic_Male',
                     'Disparity_Hispanic_Female',
                     'Disparity_Asian_Pac_Male',
                     'Disparity_Asian_Pac_Female'
        ]]
    result
    #--------------------#
    result.to_csv(f'Data/Final/201{i}_disparities.csv',index=None)
    #--------------------#